# Analyzing Spotify Songs

The aim of this work is to analyse music. Utilizing the acoustic features that Spotify provides for songs, we will look into a couple of data mining algorithms, namely popularity and genre prediction, and provide some interesting visualizations, about some fun aspects of music and some intertemporal trends.

In this work, a couple of datasets that contain information about a sample of hundreds of thousands of songs that are currently on Spotify will be used, in conjuction with the API that Spotify provides.

Author: nantoniou

## Data Source

First of all we will need to find a rich and robust data source. We will combine two; [the first one](https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db) (`SpotifyFeatures.csv`) is from Kaggle and [the second one](https://components.one/datasets/billboard-200/) (`billboard-200.db`) is from the `components.one` website.

* The first one contains 232.725 songs and has the advantage of having an equal number of songs from each genre. It contains:
 * basic song meta-data (genre, artist_name, track_name, track_id),
 * the song's popularity,
 * each song's audio features (some documentation can be found [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/)), which basically allow us to analyse the musical characteristics of the songs, such as danceability, energy, loudness and valence.

* The second one has 340.000 songs, with similar features, but it also includes some information about the each song's album and the release date of the song. However, it does not include the song's genre.

In [1]:
import pandas as pd
import sqlite3

data = pd.read_csv('SpotifyFeatures.csv')

cnx = sqlite3.connect('billboard-200.db')
data2 = pd.read_sql_query("SELECT * FROM acoustic_features", cnx)

### Data merging

We want to merge those data sources into one, but, as we can see below, the songs that are found in both of the datasets are few (only 69.884). Thus, we will use another method below.

In [43]:
len(data.loc[data.track_id.isin(data2.id)])

69884

## Data exploration

We can see that the songs go as far back as 1900!

In [42]:
data2.date.min()

'1900'

#### Song duplicates

The first data source has 91.075 duplicate songs. That is due to the fact that songs have a different record in the table for every genre they belong in.

The second data source has zero duplicated rows.

In [8]:
data.track_id.duplicated(keep=False).sum()

91075

In [7]:
data2.id.duplicated(keep=False).sum()

0

#### Albums

Let us also take a look into the albums database, that the second data source provides.

In [11]:
albums = pd.read_sql_query("SELECT * FROM albums", cnx)
albums.drop(0, inplace=True)
albums

,id,date,artist,album,rank,length,track_length
1,2,2019-01-19,A Boogie Wit da Hoodie,Hoodie SZN,1,20.0,185233.800000
2,3,2019-01-19,21 Savage,I Am > I Was,2,15.0,211050.733333
3,4,2019-01-19,Soundtrack,Spider-Man: Into The Spider-Verse,3,13.0,190866.384615
4,5,2019-01-19,Meek Mill,Championships,4,19.0,219173.894737
5,6,2019-01-19,Post Malone,beerbongs & bentleys,5,18.0,214113.611111
...,...,...,...,...,...,...,...
573942,573943,1963-01-05,The Dave Brubeck Quartet,The Dave Brubeck Quartet At Carnegie Hall,146,12.0,527888.583333
573943,573944,1963-01-05,Woody Herman,Encore: Woody Herman - 1963,147,NaN,NaN
573944,573945,1963-01-05,Lawrence Welk,1963's Early Hits,148,NaN,NaN
573945,573946,1963-01-05,Rusty Warren,Knockers Up!,149,NaN,NaN


Only 339.855 out of 573.946 albums have songs in the songs table.

In [12]:
data2.loc[data2.album.isin(albums.album)]

,id,song,album,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,date
0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.0555,0.754,142301.0,0.663,0.000000,6.0,0.101,-6.311,0.0,0.4270,90.195,4.0,0.207,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.2920,0.860,152829.0,0.418,0.000000,7.0,0.106,-9.061,0.0,0.1580,126.023,4.0,0.374,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.1530,0.718,215305.0,0.454,0.000046,8.0,0.116,-9.012,1.0,0.1270,89.483,4.0,0.196,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
3,1wJRveJZLSb1rjhnUHQiv6,Swervin (feat. 6ix9ine),Hoodie SZN,A Boogie Wit da Hoodie,0.0153,0.581,189487.0,0.662,0.000000,9.0,0.111,-5.239,1.0,0.3030,93.023,4.0,0.434,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
4,0jAfdqv18goRTUxm3ilRjb,Startender (feat. Offset and Tyga),Hoodie SZN,A Boogie Wit da Hoodie,0.0235,0.736,192779.0,0.622,0.000000,6.0,0.151,-4.653,0.0,0.1330,191.971,4.0,0.506,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339850,1EU8l9SctgP0gwIFxdjKPA,It'S A Raggy Waltz - Live,The Dave Brubeck Quartet At Carnegie Hall,The Dave Brubeck Quartet,0.6550,0.445,434427.0,0.544,0.000325,8.0,0.235,-11.662,1.0,0.0792,176.723,3.0,0.654,4My0KPjdtzCUfFjToOCiPh,1963
339851,4a0J3zWWe5IXdwWWQSypjq,King For A Day - Live,The Dave Brubeck Quartet At Carnegie Hall,The Dave Brubeck Quartet,0.8350,0.600,375533.0,0.172,0.865000,8.0,0.421,-22.897,1.0,0.0672,135.005,4.0,0.458,4My0KPjdtzCUfFjToOCiPh,1963
339852,28VEEbzNdg6r5gQFY5wWI3,Castilian Drums - Live,The Dave Brubeck Quartet At Carnegie Hall,The Dave Brubeck Quartet,0.1920,0.459,861933.0,0.606,0.612000,10.0,0.398,-13.427,0.0,0.0645,116.325,4.0,0.359,4My0KPjdtzCUfFjToOCiPh,1963
339853,7i3NXBP12BJoerk6sAATx0,Blue Rondo A La Turk - Live,The Dave Brubeck Quartet At Carnegie Hall,The Dave Brubeck Quartet,0.5220,0.444,761573.0,0.508,0.025000,10.0,0.929,-11.111,1.0,0.0526,62.879,4.0,0.592,4My0KPjdtzCUfFjToOCiPh,1963


As another sample statistic, we can see that the mean song popularity is 41.

In [20]:
data.popularity.mean()

41.12750241701579

As discussed above, merging the two data sources greatly reduces our dataset's records.

In [23]:
data2.merge(data, left_on='id', right_on='track_id')

,id,song,album,artist,acousticness_x,danceability_x,duration_ms_x,energy_x,instrumentalness_x,key_x,...,energy_y,instrumentalness_y,key_y,liveness_y,loudness_y,mode_y,speechiness_y,tempo_y,time_signature_y,valence_y
0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.0555,0.754,142301.0,0.6630,0.000000,6.0,...,0.6630,0.000000,F#,0.1010,-6.311,Minor,0.4270,90.195,4/4,0.2070
1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.2920,0.860,152829.0,0.4180,0.000000,7.0,...,0.4180,0.000000,G,0.1060,-9.061,Minor,0.1580,126.023,4/4,0.3740
2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.1530,0.718,215305.0,0.4540,0.000046,8.0,...,0.4540,0.000046,G#,0.1160,-9.012,Major,0.1270,89.483,4/4,0.1960
3,1wJRveJZLSb1rjhnUHQiv6,Swervin (feat. 6ix9ine),Hoodie SZN,A Boogie Wit da Hoodie,0.0153,0.581,189487.0,0.6620,0.000000,9.0,...,0.6620,0.000000,A,0.1110,-5.239,Major,0.3030,93.023,4/4,0.4340
4,3L19besdNQzd342qL78xqm,Demons and Angels (feat. Juice WRLD),Hoodie SZN,A Boogie Wit da Hoodie,0.0114,0.810,214593.0,0.5500,0.000006,2.0,...,0.5500,0.000006,D,0.1070,-7.460,Minor,0.1700,76.503,4/4,0.1830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69879,2TphHnGFkdWgARmEz2aEM9,Tosca / Act 3: Introduzione all'aria ... E luc...,Giacomo Puccini: Tosca,Leontyne Price,0.9270,0.194,249827.0,0.0926,0.376000,4.0,...,0.0926,0.376000,E,0.0881,-23.493,Minor,0.0395,86.987,4/4,0.0377
69880,6sA1rk1FILT7DhnrQcD5AA,Not For Me - Remastered 2002,18 Yellow Roses,Bobby Darin,0.5260,0.653,142560.0,0.4120,0.000000,8.0,...,0.4120,0.000000,G#,0.0714,-11.645,Major,0.0485,116.642,4/4,0.7180
69881,6sA1rk1FILT7DhnrQcD5AA,Not For Me - Remastered 2002,18 Yellow Roses,Bobby Darin,0.5260,0.653,142560.0,0.4120,0.000000,8.0,...,0.4120,0.000000,G#,0.0714,-11.645,Major,0.0485,116.642,4/4,0.7180
69882,6jjXwmG1czQP9krzlFsaDw,Stop the Wedding,Etta James Top Ten,Etta James,0.8500,0.416,166661.0,0.3530,0.000178,10.0,...,0.3530,0.000178,A#,0.2870,-9.482,Major,0.0400,173.064,3/4,0.6650


## Spotipy

In order to, on the one hand, use a data source with many records and, on the other hand, use a rich data source with many features, we will utilize the Spotipy API to fetch additional information about every song present in the first dataset, discarding the second, since we won't be needing its additional features.

First of all we connect to Spotipy with the credentials given to us by Spotify for Developers.

In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="XXXXXXXXXXXXXXXXXXXXXXXXX" 
secret = "XXXXXXXXXXXXXXXXXXXXXXX"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Sample query

We test the connection by fetching information about the first record in the dataset.

In [54]:
json = sp.track(data.track_id[0])
json['album']['release_date']

'2009-04-06'

### Starting the Crawiling

Spotipy allows us to request information for up to 50 songs per query, so we divide the dataset into chunks of 50.

In [85]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

ids = list(chunks(data.track_id.to_list(), 50))

#### Information quering

Below is the code that fetches the additional information for every record. We fetch:
* the "explicit" boolean feature (dictating if a song contains inappropriate words),
* the release date and
* the name of the album

In [121]:
explicit = {}
dates = {}
album_names = {}

for i in range(len(ids)):
    if i%50==0: #status update
        print(i, '/', len(ids))
    jsons = sp.tracks(ids[i])['tracks']
    for json in jsons:
        tr_id = json['id']
        explicit[tr_id] = json['explicit']
        album_names[tr_id] = json['album']['name']
        dates[tr_id] = json['album']['release_date']

0 / 4655
50 / 4655
100 / 4655
150 / 4655
200 / 4655
250 / 4655
300 / 4655
350 / 4655
400 / 4655
450 / 4655
500 / 4655
550 / 4655
600 / 4655
650 / 4655
700 / 4655
750 / 4655
800 / 4655
850 / 4655
900 / 4655
950 / 4655
1000 / 4655
1050 / 4655
1100 / 4655
1150 / 4655
1200 / 4655
1250 / 4655
1300 / 4655
1350 / 4655
1400 / 4655
1450 / 4655
1500 / 4655
1550 / 4655
1600 / 4655
1650 / 4655
1700 / 4655
1750 / 4655
1800 / 4655
1850 / 4655
1900 / 4655
1950 / 4655
2000 / 4655
2050 / 4655
2100 / 4655
2150 / 4655
2200 / 4655
2250 / 4655
2300 / 4655
2350 / 4655
2400 / 4655
2450 / 4655
2500 / 4655
2550 / 4655
2600 / 4655
2650 / 4655
2700 / 4655
2750 / 4655
2800 / 4655
2850 / 4655
2900 / 4655
2950 / 4655
3000 / 4655
3050 / 4655
3100 / 4655
3150 / 4655
3200 / 4655
3250 / 4655
3300 / 4655
3350 / 4655
3400 / 4655
3450 / 4655
3500 / 4655
3550 / 4655
3600 / 4655
3650 / 4655
3700 / 4655
3750 / 4655
3800 / 4655
3850 / 4655
3900 / 4655
3950 / 4655
4000 / 4655
4050 / 4655
4100 / 4655
4150 / 4655
4200 / 4655
425

Below we can see the contents of two of the dictionaries.

In [120]:
album_names

{'0BRjO6ga9RKCKjfDqeFgWV': 'Triple Best Of',
 '0BjC1NfoEOOusryehmNudP': 'Martin & les fées (les chansons)',
 '0CoSDzoNIKCRs124s9uTVy': 'Tears',
 '0Gc6TVm52BwZD07Ki6tIvf': "20 chansons d'or",
 '0IuslXpMROHdEPvSl1fTQK': 'Clem (Bande originale de la série)',
 '0Mf1jKa8eNAf1a4PwTbizj': 'Henri Salvador : Les débuts 1943 - 1950',
 '0NUiKYRd6jt1LKMYGkUdnZ': 'Martin & les fées (le conte)',
 '0PbIF9YVD505GutwotpB5C': 'Native Song Book Volume 1',
 '0ST6uPfvaPpJLtQwhE6KfC': 'Balanga Na 102',
 '0VSqZ3KStsjcfERGdcWpFO': 'Les génériques des dessins animés de notre enfance (40 titres)',
 '0XKgegoxLcIihK3Klpfo3N': 'Brahms : Symphonies Nos 2 & 4',
 '0hprxsuRM5vVCOfaM7l3gQ': 'Maverick - Original Motion Picture Score',
 '0jF6HUm18fg6QQCLHhfhC0': 'Les Plus Grands Hits Français',
 '0jIY0oRAp1T4mezDyEhOad': 'Yamuna Jal',
 '0pXwl2CRP5awxHsF9eET3L': 'Keys of Love',
 '0uWUjxM7oDPKpb3T2y3oZm': 'Le Radio Théâtre, Maurice Leblanc: Les aventures d\'Arsène Lupin, "813" (1961)',
 '0vS7Zid3q0dMyX8uMnlQ8s': "Les Dieux

In [134]:
explicit

{'0BRjO6ga9RKCKjfDqeFgWV': False,
 '0BjC1NfoEOOusryehmNudP': False,
 '0CoSDzoNIKCRs124s9uTVy': False,
 '0Gc6TVm52BwZD07Ki6tIvf': False,
 '0IuslXpMROHdEPvSl1fTQK': False,
 '0Mf1jKa8eNAf1a4PwTbizj': False,
 '0NUiKYRd6jt1LKMYGkUdnZ': False,
 '0PbIF9YVD505GutwotpB5C': False,
 '0ST6uPfvaPpJLtQwhE6KfC': False,
 '0VSqZ3KStsjcfERGdcWpFO': False,
 '0XKgegoxLcIihK3Klpfo3N': False,
 '0hprxsuRM5vVCOfaM7l3gQ': False,
 '0jF6HUm18fg6QQCLHhfhC0': False,
 '0jIY0oRAp1T4mezDyEhOad': False,
 '0pXwl2CRP5awxHsF9eET3L': False,
 '0uWUjxM7oDPKpb3T2y3oZm': False,
 '0vS7Zid3q0dMyX8uMnlQ8s': False,
 '0x8xSaoSfQkOYUnG1nbga0': False,
 '113pHPsGwyeklEjLhnB3ZT': False,
 '12ZjoNweObu1k9rK4SOSdL': False,
 '12hJJLMMbJDK0F9I3g5SLv': False,
 '148uA2Y78xfxDiCNjgZA0v': False,
 '14K25Ks5fdHjHfpIYOTc4y': False,
 '15CpJP0LXchBUjpB8RKh8m': False,
 '15nBPcHoH2fZmRiFd2ZhMh': False,
 '16UTNMhXjeJa8IG9uBijxZ': False,
 '16hVzfu0IJZmP3Bj4x1BB1': False,
 '17c4utCC03TPtTHXw2EVGB': False,
 '1J5pa9cW0uCsNrFwag0jDD': False,
 '1KIKeCTNJKYp

Adding the new features to our DataFrame.

In [135]:
data['explicit'] = [explicit[x] for x in data.track_id]
data['album_release'] = [dates[x] for x in data.track_id]
data['album_name'] = [album_names[x] for x in data.track_id]

In [142]:
data.sample(1)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit,album_release,album_name
14985,Dance,P!nk,Get the Party Started,2u1hMwcB9TwziV6P7jdyxX,66,0.00127,0.806,191667,0.902,0.0,...,0.212,-3.387,Minor,0.0475,128.943,4/4,0.961,False,2001-01-01,M!ssundaztood


Pandas did not automatically cast the release date into a datetime type, so we will need to do that manually.

In [144]:
data.album_release.dtype

dtype('float64')

In [163]:
data['album_release'] = pd.to_datetime(data['album_release'])

Correcting some invalid values, by looking up the correct ones.

In [161]:
min = data.album_release.min()
data.loc[data.album_release.values == min, 'album_release'] = [2017]*5

Checking DataFrame's data completeness.

In [167]:
data.isnull().sum()

genre               0
artist_name         0
track_name          0
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
explicit            0
album_release       0
album_name          0
dtype: int64

Exporting our work into a CSV file.

In [172]:
data.to_csv('songs.csv', index=False)

# Data Mining

We have our dataset ready, so we move forward with the analysis.

### Sound Wave Popularity

The first data analysis application will be popularity prediction; a common application among song analytics. The best predictors, when it comes to song popularity, are probably the artist, the label and the genre. With that being said, we will use a different approach. Wouldn't it be interesting to research and find the fundamental, inherent, *musical* characteristics that make a song popular?

Yes, we know that if Kanye West releases a song it will be popular, but it would be more interesting to know if, for example, a song that has a higher danceability is destined for success.

In [1]:
data = pd.read_csv('songs.csv')

### Linear Regression

Starting simple, aiming to not only predict a single number, but realise the musical dynamics between musical features and popularity, we will use a simple linear regression model. It will allow us to see which features impact the overall score the most.

In [42]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics as sgr
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from statsmodels.stats.outliers_influence import summary_table

%matplotlib inline

In [43]:
mod = smf.ols('popularity ~ acousticness + danceability + energy'
              '+ instrumentalness + liveness + loudness + speechiness + tempo + valence', data=data)
model = mod.fit()

### Regression results

We can see that the $R^2$ value is a bit low, but it is enough for us to reach some conclusions, in conjuction with the low P-values.

Keeping in mind that all the values are numbers between 0 and 1, the danceability seems to be the feature that *increases* popularity the most. It is a logical result, since a song that makes us dance will probably be used more in clubs, bars and parties, and will be shared more among social circles and social media.

On the contrary, we can see how a high acousticness score can greatly *reduce* the popularity. So, songs with artificial, electrical sounds (like electric guitars, drums and auto-tuned vocals) are more popular.

In [44]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     7885.
Date:                Tue, 07 Jan 2020   Prob (F-statistic):               0.00
Time:                        01:29:01   Log-Likelihood:            -9.7436e+05
No. Observations:              232725   AIC:                         1.949e+06
Df Residuals:                  232715   BIC:                         1.949e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           56.1133      0.334    168.025      0.000      55.459      56.768
acousticness       -11.9865      0.155    -77.511      0.000     -12.290     -11.683
danceability        17.6225      0.242     72.883      0.000      17.149      18.096
energy              -5.6151      0.279    -20.093      0.000      -6.163      -5.067
instrumentalness    -4.2721      0.133    -32.123      0.000      -4.533      -4.011
liveness            -9.6241      0.201    -47.856      0.000     -10.018      -9.230
loudness             0.7152      0.011     63.495      0.000       0.693       0.737
speechiness         -8.1187      0.230    -35.316      0.000      -8.569      -7.668
tempo               -0.0045      0.001     -4.031      0.000      -0.007      -0.002
valence            -13.3513      0.165    -80.753      0.000     -13.675     -13.027
==============================================================================
Omnibus:                     4295.836   Durbin-Watson:                   0.447
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4535.133
Skew:                          -0.341   Prob(JB):                         0.00
Kurtosis:                       2.953   Cond. No.                     1.67e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Feature Selection

Provided that we limit the possible features, that will be used in the model, to musical ones, which of those help us achieve the best possible prediction? That is what we will try to find in the below- adopted- cell, by using all the possible combinations of those features and trying to find the combination that maximizes the adjusted $R^2$.

In [7]:
def process_subset(y, data, feature_set):
    X = data.loc[:, feature_set].values
    X = sm.add_constant(X)
    names = ['intercept']
    names.extend(feature_set)
    model = sm.OLS(y, X)
    model.data.xnames = names
    regr = model.fit()
    return regr

import itertools

def get_best_of_k(y, data, k):
    
    best_rsquared = 0
    best_model = None
    for comb in itertools.combinations(data.columns, k):
        regr = process_subset(y, data, comb)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr

    return best_model

def best_subset_selection(data, exog):
    best_model = None
    best_models = []
    y = data.loc[:, exog]
    endog = [ x for x in data.columns if x != exog ]
    X = data.loc[:, endog]

    for i in range(1, len(data.columns)):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model = get_best_of_k(y, X, i)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(model.model.data.xnames[1:])
        best_models.append(model)

    print(f'Fitted {2**len(data.columns)} models')
    return best_model, best_models

In the first model, we arbitrarily selected the feature combination that includes all the possible features and it turns out that it is the best one.

In [8]:
best_model, _ = best_subset_selection(data.iloc[:,[4,5,6,8,9,11,12,14,15,17]], 'popularity')

Finding the best model for 1 variable
['acousticness']
Finding the best model for 2 variables
['liveness', 'loudness']
Finding the best model for 3 variables
['acousticness', 'energy', 'loudness']
Finding the best model for 4 variables
['acousticness', 'energy', 'liveness', 'loudness']
Finding the best model for 5 variables
['acousticness', 'danceability', 'liveness', 'loudness', 'valence']
Finding the best model for 6 variables
['acousticness', 'danceability', 'liveness', 'loudness', 'speechiness', 'valence']
Finding the best model for 7 variables
['acousticness', 'danceability', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']
Finding the best model for 8 variables
['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']
Finding the best model for 9 variables
['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
Fitted 1024 models


Which means that the summary of the best model, found below, is the same as the one analysed above.

In [9]:
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     7885.
Date:                Mon, 06 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:22:16   Log-Likelihood:            -9.7436e+05
No. Observations:              232725   AIC:                         1.949e+06
Df Residuals:                  232715   BIC:                         1.949e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept           56.1133      0.334    168.025      0.000      55.459      56.768
acousticness       -11.9865      0.155    -77.511      0.000     -12.290     -11.683
danceability        17.6225      0.242     72.883      0.000      17.149      18.096
energy              -5.6151      0.279    -20.093      0.000      -6.163      -5.067
instrumentalness    -4.2721      0.133    -32.123      0.000      -4.533      -4.011
liveness            -9.6241      0.201    -47.856      0.000     -10.018      -9.230
loudness             0.7152      0.011     63.495      0.000       0.693       0.737
speechiness         -8.1187      0.230    -35.316      0.000      -8.569      -7.668
tempo               -0.0045      0.001     -4.031      0.000      -0.007      -0.002
valence            -13.3513      0.165    -80.753      0.000     -13.675     -13.027
==============================================================================
Omnibus:                     4295.836   Durbin-Watson:                   0.447
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4535.133
Skew:                          -0.341   Prob(JB):                         0.00
Kurtosis:                       2.953   Cond. No.                     1.67e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Decision Tree Regressor

We will continue looking into the same application, but we will change into a more advanced predictory model; the Decision Tree Regressor. The application is more complex than we gave it credit for, above. The tree will allow us to handle the multi co-linearity better, while also allowing us to have some basic form of model visualization, just for the sake of it; it is supposed to be a fun work after all!

In [12]:
columns = data.iloc[:, [4,5,6,8,9,11,12,14,15,17]]
columns

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0,0.61100,0.389,0.910,0.000000,0.3460,-1.828,0.0525,166.969,0.814
1,1,0.24600,0.590,0.737,0.000000,0.1510,-5.559,0.0868,174.003,0.816
2,3,0.95200,0.663,0.131,0.000000,0.1030,-13.879,0.0362,99.488,0.368
3,0,0.70300,0.240,0.326,0.000000,0.0985,-12.178,0.0395,171.758,0.227
4,4,0.95000,0.331,0.225,0.123000,0.2020,-21.150,0.0456,140.576,0.390
...,...,...,...,...,...,...,...,...,...,...
232720,39,0.00384,0.687,0.714,0.544000,0.0845,-10.626,0.0316,115.542,0.962
232721,38,0.03290,0.785,0.683,0.000880,0.2370,-6.944,0.0337,113.830,0.969
232722,47,0.90100,0.517,0.419,0.000000,0.0945,-8.282,0.1480,84.135,0.813
232723,44,0.26200,0.745,0.704,0.000000,0.3330,-7.137,0.1460,100.031,0.489


We will split the dataset into 80%-20%, train and test respectively.

In [3]:
import sklearn as sk
from sklearn import metrics
import numpy as np
from sklearn.model_selection import train_test_split

X, y = columns.iloc[:, 1:], columns.popularity

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, shuffle=True)

We will use Mean Absolute Error as the criterion and we will limit the fit to a maximum depth of 5, using only the first 100.000 records, allowing for a fitting in a resonable time, with our limited resources.

In [7]:
from sklearn.tree import DecisionTreeRegressor

tree_rgr = DecisionTreeRegressor(criterion='mae', max_depth=5)

tree_rgr.fit(X_train[:100000], y_train[:100000])

DecisionTreeRegressor(criterion='mae', max_depth=5, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Below we will calculate the MAE of our model.

In [ ]:
pred = tree_rgr.predict(X_test)
mae = metrics.mean_absolute_error(y_test, pred)

Exporting the tree into an image.

In [ ]:
from sklearn import tree
from sklearn.externals.six import StringIO
from IPython.display import Image  
from IPython.display import display
import pydotplus
import graphviz

dot_data = StringIO()  
tree.export_graphviz(tree_rgr, out_file=dot_data,  
                     feature_names=columns.columns[1:],  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png()) 

graph = graphviz.Source(tree.export_graphviz(tree_rgr, max_depth=3, feature_names=columns.columns[1:], rounded=True,   filled=True, special_characters=True))

graph.format = "png"
graph.render("graph")

Below we can see our tree.
<img src='images/graph.png'/>

Comparing our model's MAE to a baseline; just guessing the mean popularity.

In [48]:
print(metrics.mean_absolute_error(y_test, tree_rgr.predict(X_test)))
print(metrics.mean_absolute_error(y_test, [y_train.mean()]*len(y_test)))

0.23407699868689225
14.84425210294308


Using a random value to see what our model predicts.

In [37]:
from random import randrange
rand = randrange(len(X_test))
tree_rgr.predict([X_test[rand]])[0]

45.0

In [38]:
y_test[rand]

44

## Genre Prediction

The second application we will work on is genre prediction. This time we will use all of the available features, with the goal of classifying a song into one- or more- genres. To do that we will utilise a neural network, since they fare well with multi-label classification applications.

Below we will correct some genre data and, also, turn our categorical variables into k-hot encoded features.

In [1]:
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

data = pd.read_csv('songs.csv')
data.loc[data.genre=='Children’s Music', 'genre'] = 'Children\'s Music'
data = pd.get_dummies(data, columns=['mode', 'explicit'], drop_first=True)

We use MultiLabelBinarizer to create a 26-hot encoded DataFrame with the genres that a song belongs to.

In [3]:
mlb = MultiLabelBinarizer()
classes = mlb.fit_transform([[x] for x in data.genre])

X, y = data.iloc[:, [4,5,6,7,8,9,11,12,13,14,16,19,20]], classes
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, shuffle=True)
X.columns

Index(['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'mode_Minor', 'explicit_True'],
      dtype='object')

A simple network will be used.
* The first layer is a Dense one, of 32 neurons, with each one of those having the 13 features as input.
* Next there is the dropout layer, to limit over-fitting.
* Thirdly, there is a second Dense layer, of 16 neurons. The Dense layers seemed to provide better results with the relu activation function.
* Lastly we have the classification layer, with 26 neurons (the number of classes we try to predict) and a sigmoid activation function, since we have a multi label classification application. For the same reason, the loss function is binary crossentropy.

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model_simple = Sequential([
    Dense(32, input_shape=(13,), activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(NUM_CLASSES, activation='sigmoid')
])

model_simple.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_simple.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                448       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 26)                442       
Total params: 1,418
Trainable params: 1,418
Non-trainable params: 0
_________________________________________________________________


To understand the network's structure better, we can see it visualized below. 20% of the neuron connections between the two hidden layers are discarded in each epoch (due to the Dropout).

<img src='images/network_pic.png' width="500">

The accuracy may seem quite high, but it is skewed because of the many zeroes present in the prediction. Basically, if we guess that a song does not belong in any class we are more than 90% correct, since it may belong to one or two genres, out of the twenty six.

In [19]:
simple_history = model_simple.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Train on 186180 samples, validate on 46545 samples
Epoch 1/2
186180/186180 [==============================] - 79s 423us/sample - loss: 108.8968 - accuracy: 0.9261 - val_loss: 0.3083 - val_accuracy: 0.9322
Epoch 2/2
186180/186180 [==============================] - 62s 332us/sample - loss: 0.2540 - accuracy: 0.9580 - val_loss: 0.1613 - val_accuracy: 0.9615


### Network Performance

To improve our assesment of the network's performance, we will use the test dataset again. The predictions are 26 numbers [0, 1], describing how probable it is for that song to belong in that specific genre. Thus, we will change them into binaries, by using a treshold, accepting predictions with a higher number that the treshold as actual genres of that song.

In [32]:
treshold = 0.05
pred_test_std = model_simple.predict(X_test)

pred_ar = pred_test_std[0]
print(pred_ar)
for position, prediction in enumerate(pred_ar):
    if prediction > treshold:
        pred_ar[position] = 1
    else:
        pred_ar[position] = 0
print(pred_ar)

[0.00260785 0.04031795 0.038791   0.03842062 0.06280309 0.03958815
 0.04268703 0.03735015 0.03785449 0.04189181 0.04028147 0.04109302
 0.04069251 0.04147059 0.03397501 0.03635618 0.04104292 0.03943545
 0.03880572 0.0384241  0.03909135 0.04151443 0.03918499 0.04091018
 0.04250628 0.0393123 ]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


# Visualizing Music

We will explore some visualizations of our data below.

*Note:* I wanted to experiment with PowerBI, so the visualizations are prepared there.

### Rock Titles
We will start off with a visualization of the songs' titles, focusing on titles from Rock songs.

We will read the data, correct the wrong genre data and then split the titles into words.

In [2]:
data = pd.read_csv('songs.csv')
data.loc[data.genre=='Children’s Music', 'genre'] = 'Children\'s Music'

words = data.loc[data.genre=='Rock'].track_name.str.split()

reshaped_words = np.reshape(words.tolist(), (1,-1))[0]

There is no need to keep the words from each title separate.

In [6]:
words=[]
for word in reshaped_words:
    words += word

Removing the stopwords.

In [7]:
from nltk.corpus import stopwords
filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]

Getting the number each specific word was used in all of the available titles.

In [8]:
wors = pd.Series(filtered_words).value_counts(ascending=False)

Manually removing words that do not add something to our visualization.

In [30]:
wors.drop('2011', inplace=True)

In [32]:
wors.head(10)#.to_csv('words_rock.csv')

Love     336
Live     111
Man       95
One       84
Good      83
Night     79
Life      79
Time      78
Song      77
Go        75
dtype: int64

Having prepared the list with the most common words, and their respective number of uses, it is easy to create a word cloud on PowerBI, as seen below.

<img src='images/rock.png'>

We can do the same for any other genre.

In [69]:
data.genre.unique()

array(['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Rap', 'Indie', 'Classical', 'Pop', 'Reggae',
       'Reggaeton', 'Jazz', 'Rock', 'Ska', 'Comedy', 'Soul', 'Soundtrack',
       'World'], dtype=object)

### Additional Visualizations

I experimented with an SQL DB, which was used to make the below visualizations, but they were interesting enough to be included in this work.

#### Valence & Danceability
We can see the average valence and danceability, by decade (*note:* the records from the 1880s, 1890s and the 1900s are few).<br>They seem to be following the same trends, with an interesting observation beeing the two major dips, during the two World Wars.
<img src='images/danc_val.png' width='750'>

#### Energy & Loudness

Similarly to Valence & Danceability, the Energy & the Loudness of a song follow similar trends. Granted, those two features are highly correlated, but energy, besides loudness, describes dynamic range, timbre, onset rate, and general entropy.

We can see once again the effect the two World Wars had on music. However, even though there were some dips on those two metrics during WWI and WWII, currently they seem to be at their highest level, proving the fact that people nowadays are mostly interested about loud, fast and sense-hightening, dopamine-releasing music.

<img src='images/energy.png'>

#### Top Artists
The artists that had the highest average popularity, for the 4 most popular genres (provided that they had at least 5 songs in that genre).

<img src='images/topArtists.png'>

#### Computerizing Music

The average acousticness seems to be steadily dropping. As we saw above, the acousticness score seems to be reducing the popularity score greatly, with people prefering artificial sounds, so limiting acousticness is to be expected from artists.

<img src='images/acousticness.png'>